In [1]:
#%cd drive/MyDrive/Colab_Notebooks/Observation-earth/

In [2]:
#!git clone https://github.com/louisPoulain/IPEO-Understanding-the-Amazon-from-Space.git

In [3]:
#%cd IPEO-Understanding-the-Amazon-from-Space/

In [4]:
#!git checkout Test_Louis

In [5]:
#!pip install -q -U pytorch_lightning tensorboard

In [6]:
#from google.colab import drive
#drive.mount('/content/drive')

In [1]:
%cd drive/MyDrive/Colab_Notebooks/Observation-earth/IPEO-Understanding-the-Amazon-from-Space/
!git checkout Test_Louis
!git pull

/content/drive/MyDrive/Colab_Notebooks/Observation-earth/IPEO-Understanding-the-Amazon-from-Space
M	__pycache__/accuracy_metrics.cpython-38.pyc
M	__pycache__/blocks.cpython-38.pyc
M	main_louis.ipynb
Already on 'Test_Louis'
Your branch is up to date with 'origin/Test_Louis'.
remote: Enumerating objects: 17, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 14 (delta 4), reused 13 (delta 3), pack-reused 0
Unpacking objects: 100% (14/14), done.
From https://github.com/louisPoulain/IPEO-Understanding-the-Amazon-from-Space
   374c6b7..f8bf717  Test_Louis -> origin/Test_Louis
Updating 374c6b7..f8bf717
Fast-forward
 __pycache__/accuracy_metrics.cpython-310.pyc | Bin 0 -> 1394 bytes
 __pycache__/blocks.cpython-310.pyc           | Bin 0 -> 3772 bytes
 __pycache__/dataset.cpython-310.pyc          | Bin 0 -> 4675 bytes
 __pycache__/model.cpython-310.pyc            | Bin 0 -> 4720 bytes
 create_labels_dataframe.ipynb                |

In [2]:
!pip install -q -U pytorch_lightning
!pip install pandas==1.5.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.3/800.3 KB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.4/125.4 KB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.4/512.4 KB 44.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 55.3 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 1.3.5
    Uninstalling pandas-1.3.5:
      Successfully uninstalled pandas-1.3.5


In [3]:
from dataset import DatasetAmazon
from model import PlanetModel, testModel, ResNet
import torch
import pytorch_lightning as pl
from pytorch_lightning import loggers as pl_loggers
import time

from pytorch_lightning.callbacks import ModelCheckpoint
import os
import sys
#sys.path.append("drive/MyDrive/Colab_Notebooks/Observation-earth/IPEO-Understanding-the-Amazon-from-Space/")
dir_path = "drive/MyDrive/Colab_Notebooks/Observation-earth/IPEO-Understanding-the-Amazon-from-Space"

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [5]:
full, tiny = True, False
train_dataset = DatasetAmazon(full=full, tiny=tiny, path_to_labels="CSV/train_label_vector.pkl")
val_dataset = DatasetAmazon(full=full, tiny=tiny, val=True, path_to_labels="CSV/train_label_vector.pkl")
print(len(train_dataset), len(val_dataset))

loading took  0.9652674198150635
loading took  0.23548173904418945
24285 6064


In [6]:
pretrained = True
depth = 18
test_model = ResNet(depth=depth) if pretrained else testModel(max_channels=512) 
model = PlanetModel(model=test_model)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [7]:
csv_logger = pl_loggers.CSVLogger(save_dir="", name = "logs")

#checkpoint_callback = ModelCheckpoint(
#    dirpath="checkpoints",#dir_path + 
#    filename='test-{epoch}-{val_accuracy:.2f}',
#    monitor="val_accuracy",#val_loss
#    mode="max"#min
#    )

max_epochs = 50

checkpoint_callback = ModelCheckpoint(
    dirpath="../IPEO_Planet_project/checkpoints",
    filename=f'ResNet{depth}'+'-{epoch}_'+f'over_{max_epochs}'+'-{val_accuracy:.2f}',
    monitor="val_accuracy",
    mode="max"
    ) if pretrained else ModelCheckpoint(
    dirpath="../IPEO_Planet_project/checkpoints",
    filename='test-{epoch}_'+ f'over_{max_epochs}'+ '-{val_accuracy:.2f}',
    monitor="val_accuracy",
    mode="max"
    )

In [8]:
trainer = pl.Trainer(max_epochs=max_epochs, accelerator="gpu", devices=[0], 
                     logger=csv_logger, callbacks=[checkpoint_callback],
                     resume_from_checkpoint=None, check_val_every_n_epoch=1)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [9]:
batch_size = 128 if full else (3 if tiny else 20)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, persistent_workers=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4, persistent_workers=True)

In [10]:
t = time.time()
trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=val_loader)
training_time = time.time()-t
print(f"Training time: {training_time}s")

/usr/local/lib/python3.8/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:604: UserWarning: Checkpoint directory /content/drive/MyDrive/Colab_Notebooks/Observation-earth/IPEO_Planet_project/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type        | Params
--------------------------------------
0 | model | ResNet      | 11.2 M
1 | loss  | custom_loss | 0     
--------------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.753    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=50` reached.


Training time: 3109.819226026535s


In [11]:
from accuracy_metrics import Hamming_distance, transform_pred, overall_acc

def count_false(y_pred, y, threshold=0.65):
    y_pred = transform_pred(y_pred, threshold)
    false_positive = torch.zeros(y_pred.shape)
    false_negative = torch.zeros(y_pred.shape)
    false_positive += torch.where((y_pred-y)==1, 1, 0) # predict 1 when you should predict 0
    false_negative += torch.where((y_pred-y)==-1, 1, 0) # predict 0 but should have predicted 1
    return false_positive.numpy(), false_negative.numpy()

In [12]:
test_dataset = DatasetAmazon(full=full, test=True, path_to_labels="CSV/train_label_vector.pkl")
test_dl = torch.utils.data.DataLoader(test_dataset, batch_size = 1, num_workers=4, shuffle=False)

loading took  0.4075286388397217


In [14]:
# replace with the checkpoint of your trained model
model_name = "ResNet18-epoch=47_over_50-val_accuracy=0.93.ckpt"
checkpoint_path = "../IPEO_Planet_project/checkpoints/"+model_name
pretrained = True # beware of changing this 
depth = 18 # same
test_model = ResNet(depth=depth) if pretrained else testModel(max_channels=512) 
model = PlanetModel(model=test_model)
model.load_state_dict(torch.load(checkpoint_path)["state_dict"])
rs = trainer.predict(model, dataloaders=test_dl)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 190it [00:00, ?it/s]

In [15]:
import numpy as np
false_neg, false_pos = np.zeros((1, 17)), np.zeros((1, 17))
y_hat, y = list(map(list, zip(*rs)))
overall_accuracy = 0.0
hamming_dist = 0.0
for i in range(len(y_hat)):
    overall_accuracy += overall_acc(y_hat[i], y[i])
    hamming_dist += Hamming_distance(y_hat[i], y[i])
    false_positive, false_negative = count_false(y_hat[i], y[i])
    false_pos += false_positive
    false_neg += false_negative
overall_accuracy = overall_accuracy/len(y_hat)
hamming_dist = hamming_dist/len(y_hat)
                                        
print(f"Overall accuracy: {overall_accuracy:.4f}\nHamming distance: {hamming_dist}")
print("False positive (predict 1 instead of 0): ",false_pos, "\nFalse negative (predict 0 instead of 1): ",false_neg)

Overall accuracy: 0.9298
Hamming distance: 0.07021969556808472
False positive (predict 1 instead of 0):  [[210. 348. 115. 347.   0.   0.   0.   0.   0.   0.   0.   0.   0. 416.
    0.   0.   0.]] 
False negative (predict 0 instead of 1):  [[ 319.  379.  181.  141. 3062.   24.  212.   83.   83.   24. 1090.  916.
  1841.  100. 2044.   87.   51.]]
